# Model deployment of Fine-Tuned Llama 3.1 8B in Amazon SageMaker

---

In this demo notebook, we demonstrate how to deploy the fine-tuned model from the notebook [01_llama-3.1-8b-qlora-sft.ipynb](./01_llama-3.1-8b-qlora-sft.ipynb) in an Amazon SageMaker real-time endpoint.

---

JupyterLab Instance Type: ml.t3.medium

Install the required libriaries, including the Hugging Face libraries, and restart the kernel.

In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install -q -U boto3
%pip install -q -U botocore
%pip install -q -U Levenshtein
%pip install -q -U scikit-learn==1.5.1
%pip install -q -U seaborn


## Setup Configuration file path

We are setting the directory in which the config.yaml file resides so that remote decorator can make use of the settings through [SageMaker Defaults](https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk).

This notebook is using the Hugging Face container for the `us-east-1` region. Make sure you are using the right image for your AWS region, otherwise edit [config.yaml](./config.yaml). Container Images are available [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)


In [ ]:
import os

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## Deploy the Fine-Tuned model

In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

bucket_name = sagemaker_session.default_bucket()
job_prefix = f"train-{model_id.split('/')[-1].replace('.', '-')}-auto"

In [ ]:
def get_last_job_name(job_name_prefix):
    import boto3
    sagemaker_client = boto3.client('sagemaker')

    search_response = sagemaker_client.search(
        Resource='TrainingJob',
        SearchExpression={
            'Filters': [
                {
                    'Name': 'TrainingJobName',
                    'Operator': 'Contains',
                    'Value': job_name_prefix
                },
                {
                    'Name': 'TrainingJobStatus',
                    'Operator': 'Equals',
                    'Value': "Completed"
                }
            ]
        },
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1)

    return search_response['Results'][0]['TrainingJob']['TrainingJobName']

In [ ]:
job_name = get_last_job_name(job_prefix)

job_name

#### Inference configurations

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [ ]:
instance_count = 1
instance_type = "ml.g5.4xlarge"
number_of_gpu = 1
health_check_timeout = 700

In [ ]:
import json

model = HuggingFaceModel(
    image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.4.0-tgi2.3.1-gpu-py311-cu124-ubuntu22.04-v2.0', #image_uri with py311
    model_data=f"s3://{bucket_name}/{job_name}/{job_name}/output/model.tar.gz",
    role=get_execution_role(),
    env={
        'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
        'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
        'HF_MODEL_QUANTIZE': 'bitsandbytes',
        'MAX_INPUT_LENGTH': '4096',
        'MAX_TOTAL_TOKENS': '8192'
    }
)

In [ ]:
predictor = model.deploy(
    initial_instance_count=instance_count,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
    model_data_download_timeout=3600
)

## Model Evaluation - Fine-tuned model vs. Base model

We are going to evaluate the fine-tuned model and the base model on two metrics:
* BLEU Score
* Accuracy score with Levenshtein distance

BLEU (bilingual evaluation understudy) is an algorithm for evaluating the quality of text which has been machine-translated from one natural language to another.


Normalized Levenshtein distance is an algorithm for evaluating accuracy degree of how close the calculated or measured values are to the actual value.

### Define Model predictor

In [ ]:
from sagemaker.huggingface.model import HuggingFacePredictor

In [ ]:
endpoint_name = "<ENDPOINT_NAME>" #Required if you want to create a predictor without running the previous code

In [ ]:
if 'predictor' not in locals() and 'predictor' not in globals():
    print("Create predictor")
    predictor = HuggingFacePredictor(
        endpoint_name=endpoint_name
    )

### Define Amazon Bedrock client

In [ ]:
import boto3

In [ ]:
bedrock_client = boto3.client('bedrock-runtime')

In [ ]:
bedrock_model_id = "meta.llama3-8b-instruct-v1:0"

### Create an evaluation dataset

In [ ]:
import pandas as pd

df = pd.read_csv("./sample_dataset.csv")

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42)
train, valid = train_test_split(train, test_size=10, random_state=42)

print("Number of validation elements: ", len(valid))

### Answer generation

In [ ]:
#Let us first do a small test...
row = valid.iloc[0]
row['DESC_DEFECT']

In [ ]:
def build_defect_request_body(row, top_p=0.9, temperature=0.2, max_new_tokens=512):
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>These are the information related to the defect:\nManufacturer: {row['MFGNAME']}\nComponent: {row['COMPNAME']}\nDescription of the defect: {row['DESC_DEFECT']}\n\n\nWhat are the consequences of defect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

    return {
        'inputs': prompt,
        'parameters': {
            "top_p": top_p,
            "temperature": temperature,
            "max_new_tokens": max_new_tokens,
            "return_full_text": False,
            "stop": [
                '<|eot_id|>',
                '<|end_of_text|>'
            ]
        }
    }

In [ ]:
predictor.predict(build_defect_request_body(row))

In [ ]:
import json
import time

evaluation_set = []

for index, row in valid.iterrows():
    print("Example ", index)

    ## Generate response with the fine-tuned model
    body = build_defect_request_body(row)

    start_time = time.time()

    response = predictor.predict(body)

    end_time = time.time()

    print(f"Generated response with fine-tuned model: {end_time - start_time:.6f} seconds")

    response_fine_tuned = response[0]['generated_text'].strip()

    print(response_fine_tuned)

    ## Generate response with the base model

    prompt = f"""
        These are the information related to the defect:
        Manufacturer: {row['MFGNAME']}
        Component: {row['COMPNAME']}
        Description of a defect:
        {row['DESC_DEFECT']}
    """

    messages = [
        {
            "role": "user",
            "content": [{"text": prompt},
                        {"text": "What are the consequences?"}]
        }
    ]

    start_time = time.time()

    response = bedrock_client.converse(
        modelId=bedrock_model_id,
        messages=messages,
        inferenceConfig={
            "temperature": 0.2,
            "topP": 0.9,
            "maxTokens": 512
        }
    )

    end_time = time.time()

    print(f"Generated response with base model: {end_time - start_time:.6f} seconds")

    response_base = response['output']['message']["content"][0]["text"]
    print(response_base)

    evaluation_set.append({
        "index": index,
        "target_answer": row["CONEQUENCE_DEFECT"],
        "fine_tuned_answer": response_fine_tuned,
        "base_answer": response_base
    })

    print("******************")

with open("llama_32_1b_evaluation_dataset.json", "w") as f:
    json.dump(evaluation_set, f, indent=4)

### BLEU Score evaluation

In [ ]:
import nltk
import re

def clean_array(string):
    filtered_words = []
    
    for element in string:
        cleaned_word = re.sub(r'[^a-zA-Z]', '', element)
        if cleaned_word:
            filtered_words.append(cleaned_word)
    
    return filtered_words

def calculate_score(index, reference, hp_1, hp_2):
    reference_split = clean_array(reference.split(" "))
    
    hp_1_split = clean_array(hp_1.split(" "))
    hp_2_split = clean_array(hp_2.split(" "))
    
    BLEUscore_hp_1 = nltk.translate.bleu_score.sentence_bleu([reference_split], hp_1_split)
    BLEUscore_hp_2 = nltk.translate.bleu_score.sentence_bleu([reference_split], hp_2_split)
    print("Example ", index)
    print("Fine-tuned score: ", BLEUscore_hp_1)
    print("Base score: ", BLEUscore_hp_2)

    print("******************")

    return BLEUscore_hp_1, BLEUscore_hp_2

In [ ]:
import json

with open('llama_32_1b_evaluation_dataset.json', 'r') as file:
    data = json.load(file)

file.close()

data = []

for el in evaluation_set:
    BLEUscore_fine_tuned, BLEUscore_base = calculate_score(
        el["index"],
        el["target_answer"],
        el["fine_tuned_answer"],
        el["base_answer"])
    
    data.append([el["index"], BLEUscore_fine_tuned, BLEUscore_base])

df = pd.DataFrame(data, columns=["index", "Fine-tuned score", "Base score"])

df["Fine-tuned score"] = df["Fine-tuned score"].astype(float)
df["Base score"] = df["Base score"].astype(float)

df.to_csv("./llama_32_1b_bleu_scores.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.read_csv("llama_32_1b_bleu_scores.csv")

data1 = df['Fine-tuned score']
data2 = df['Base score']

combined_data = pd.DataFrame({
    'Fine-tuned model scores': data1,
    'Base model scores': data2
})

plt.figure(figsize=(12, 6))
sns.boxplot(data=combined_data)
plt.xlabel('Models', fontsize=12)
plt.ylabel('Score')
plt.title('Distribution of Scores: Fine-tuned vs Base Model')

plt.savefig('./images/llama_32_1b_bleu_scores.png')

plt.show()

#### BLEU Score Table

##### Example 1

Training Arguments:
* `epochs`: 1
* `per_device_train_batch_size`: 2
* `per_device_test_batch_size`: 2
* `gradient_accumulation_steps`: 2
* `gradient_checkpointing`: True

These results are obtained by fine-tuning on 6000 rows in total, where 3000 rows of the dataset were duplicated for having both therminology on the `CONEQUENCE_DEFECT` and `CORRECTIVE_ACTION`.

Total time for fine-tuning:
* `ml.g5.12xlarge`: ~42 minutes on 4 GPUs

Evaluation is performed on 10 rows extracted from the original dataset and not contained in the dataset used for the fine-tuning.

BLEU score is performed with the fine-tuned model hosted on Amazon SageMaker, with an `ml.g5.4xlarge`, and the base model in Amazon Bedrock.

Base model: `LLama-3.1 8B Instruct`

![BLEU Scores Table](./images/llama_32_1b_bleu_scores_table.png)

##### BLEU Scores graph

![BLEU Scores Table](./images/llama_32_1b_bleu_scores.png)

By comparing the scores in the "Fine-tuned Score" and "Base Score" columns, we can assess the performance improvement (or degradation) achieved by fine-tuning the model on the specific task or domain.

The analysis suggest that in most cases, the fine-tuned model seems to be outperforming the base model. The fine-tuned model appears to be more consistent in its performance.

Possible improvements:
* Examples repetition: Provide similar examples for improving further improving the vocabulary of the fine-tuned model
* Increse the number of epochs

***

Base model: `LLama-3 70B Instruct`

![BLEU Scores Table](./images/llama_32_1b_bleu_scores_table_70.png)

##### BLEU Scores graph

![BLEU Scores Table](./images/llama_32_1b_bleu_scores_70.png)

By comparing the scores in the "Fine-tuned Score" and "Base Score" columns, we can assess the performance improvement (or degradation) achieved by fine-tuning the model on the specific task or domain.

The analysis suggest that in most cases, the fine-tuned model seems to be outperforming the base model. The fine-tuned model appears to be more consistent in its performance.

Possible improvements:
* Examples repetition: Provide similar examples for improving further improving the vocabulary of the fine-tuned model
* Increse the number of epochs

***

### Accuracy evaluation

In [ ]:
import Levenshtein

def levenshtein_similarity(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    normalized_distance = 1 - (distance / max_len) if max_len > 0 else 1
    return normalized_distance

In [ ]:
import json

with open('llama_32_1b_evaluation_dataset.json', 'r') as file:
    data = json.load(file)

file.close()

data = [] 

for el in evaluation_set:
    print("Example ", el["index"])
    score_fine_tuned = levenshtein_similarity(el["fine_tuned_answer"], el["target_answer"])
    print("Fine-tune score: ", score_fine_tuned)
    score_base = levenshtein_similarity(el["base_answer"], el["target_answer"])
    print("Base score: ", score_base)
    print("******************")

    data.append([el["index"], score_fine_tuned, score_base])

df = pd.DataFrame(data, columns=["index", "Fine-tuned score", "Base score"])

df["Fine-tuned score"] = df["Fine-tuned score"].astype(float)
df["Base score"] = df["Base score"].astype(float)

df.to_csv("./llama_32_1b_levenshtein_scores.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.read_csv("llama_32_1b_levenshtein_scores.csv")

data1 = df['Fine-tuned score']
data2 = df['Base score']

combined_data = pd.DataFrame({
    'Fine-tuned model scores': data1,
    'Base model scores': data2
})

plt.figure(figsize=(12, 6))
sns.boxplot(data=combined_data)
plt.xlabel('Models', fontsize=12)
plt.ylabel('Score')
plt.title('Distribution of Scores: Fine-tuned vs Base Model')

plt.savefig('./images/llama_32_1b_levenshtein_scores.png')

plt.show()

#### Normalized Levenshtein Score Table

#### Example 1

Training Arguments:
* `epochs`: 1
* `per_device_train_batch_size`: 2
* `per_device_test_batch_size`: 1
* `gradient_accumulation_steps`: 2
* `gradient_checkpointing`: True

These results are obtained by fine-tuning on 6000 rows in total, where 3000 rows of the dataset were duplicated for having both therminology on the `CONEQUENCE_DEFECT` and `CORRECTIVE_ACTION`.

Total time for fine-tuning:
* `ml.g5.12xlarge`: ~39 minutes on 4 GPUs

Evaluation is performed on 10 rows extracted from the original dataset and not contained in the dataset used for the fine-tuning.

Levenshtein score is performed with the fine-tuned model hosted on Amazon SageMaker, with an `ml.g5.4xlarge`, and the base model in Amazon Bedrock.

Base model: `LLama-3.1 8B Instruct`

![BLEU Scores Table](./images/llama_32_1b_levenshtein_scores_table.png)

##### BLEU Scores graph

![BLEU Scores Table](./images/llama_32_1b_levenshtein_scores.png)

By comparing the scores in the "Fine-tuned Score" and "Base Score" columns, we can assess the performance improvement (or degradation) achieved by fine-tuning the model on the specific task or domain.

The analysis suggest that the fine-tuned model is clearly outperforming the base model across almost all examples. This suggests that the fine-tuning process has been quite effective in improving the model's accuracy for this specific task.

In the Normalized Levenshtein distance, the range is from 0 to 1, where closer to 0 means better performance. The fine-tuned model often achieves scores closer to 0, indicating higher accuracy.

Possible improvements:
* Examples repetition: Provide similar examples for improving further improving the vocabulary of the fine-tuned model
* Increse the number of epochs

***

Base model: `LLama-3 70B Instruct`

![BLEU Scores Table](./images/llama_32_1b_levenshtein_scores_table_70.png)

##### BLEU Scores graph

![BLEU Scores Table](./images/llama_32_1b_levenshtein_scores_70.png)

By comparing the scores in the "Fine-tuned Score" and "Base Score" columns, we can assess the performance improvement (or degradation) achieved by fine-tuning the model on the specific task or domain.

The analysis suggest that the fine-tuned model is clearly outperforming the base model across almost all examples. This suggests that the fine-tuning process has been quite effective in improving the model's accuracy for this specific task.

In the Normalized Levenshtein distance, the range is from 0 to 1, where closer to 0 means better performance. The fine-tuned model often achieves scores closer to 0, indicating higher accuracy.

Possible improvements:
* Examples repetition: Provide similar examples for improving further improving the vocabulary of the fine-tuned model
* Increse the number of epochs


#### Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)